In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import importlib


import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature



In [ ]:
sst_file = '/glade/p/cesmdata/cseg/inputdata/atm/cam/sst/sst_HadOIBl_bc_0.9x1.25_1850_2017_c180507.nc'
dHSST = xr.open_dataset(sst_file ) 
sst=dHSST.SST_cpl
sst.shape
dates=dHSST.date
print(dates.values[-1-1:-1])
print(list(dHSST.variables))

In [ ]:
drc='/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/TS/'
fils='b.e11.BRCP85C5CNBDRD.f09_g16.0**.cam.h0.TS.208101-210012.nc'
dse=xr.open_mfdataset( drc+fils ,concat_dim='ensemble',combine='nested')
lats=dse.lat.values
lons=dse.lon.values

fils='b.e11.BRCP85C5CNBDRD.f09_g16.0**.cam.h0.TS.200601-208012.nc'
dse0=xr.open_mfdataset( drc+fils ,concat_dim='ensemble',combine='nested')

drcLF='/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/LANDFRAC/'
filLF='b.e11.BRCP85C5CNBDRD.f09_g16.001.cam.h0.LANDFRAC.208101-210012.nc'
dsLF=xr.open_dataset( drcLF+filLF )
landf=dsLF.LANDFRAC.values[0,:,:]


In [ ]:
print(np.shape(landf))

In [ ]:
print(dse0.TS.shape)

In [ ]:
print( list( dse0.variables ))

In [ ]:
print( dse0.time_bnds.values[0,900-132,:])

In [ ]:
nens,nt,ny,nx = np.shape( dse.TS )

In [ ]:
tsoo=dse.TS.mean(dim='time')

In [ ]:
tsoo.shape


In [ ]:
tsoo=tsoo.values


# 'Center' the data

In [ ]:
# Don't be bothered by apparent dimension mismatch.
# This line 'broadcasts' the mean taken for axis=0
# to the other dimensions, i.e., this line is equivalent 
# to subtracting 2 3D arrays where the mean array is 
# a stack of identical 2D arrays. Thanks ChatGPT.
#-----------------------------------------------------
tsoop = tsoo - tsoo.mean(axis=0)


In [ ]:
for em in np.arange( nens ):
    tsoop[em, :,:] = np.where( landf < 0.05 , tsoop[em,:,:], 0. )

In [ ]:
# Scale with SQRT(cos(lat))
Lon,Lat=np.meshgrid(lons,lats) 
CosScale=np.sqrt( np.cos( (np.pi/180.)*Lat ) )
print(np.shape(CosScale))
plt.contour( CosScale )

In [ ]:
# More "broadcasting"
tsoog = tsoop * CosScale

In [ ]:
#plt.plot( tsoo[:,90,90]-304.)
plt.contour( tsoog[0,:,:])


In [ ]:
print(lats[[60,131]])
tsoog=tsoog[:,60:132,:]
nt,ny,nx = np.shape(tsoog)

In [ ]:
tsoog=np.reshape( tsoog, (nt, ny*nx) )

In [ ]:
u,s,vt=np.linalg.svd( tsoog, full_matrices=False )

In [ ]:
print(np.shape(u))
print(np.shape(s))
#plt.plot(np.average(u,axis=1))
plt.plot( u.flatten() )

In [ ]:
#print( np.shape(vt))
fig=plt.figure(figsize=(20,10))

lv=np.linspace( -.03,.03,num=31)
for n in np.arange(12):
    ax1 = fig.add_subplot(3, 4, n+1 ) # , projection=projPC )
    co=ax1.contourf( vt[n,:].reshape(ny,nx) , levels=lv)
    plt.colorbar(co ) #,cax=cax,orientation='horizontal')


In [ ]:
## 

In [ ]:
from sklearn.decomposition import PCA 

In [ ]:
pca=PCA(n_components=20)

In [ ]:
tsoov=pca.fit_transform(tsoop)

In [ ]:
print(np.shape(pca.components_))

In [ ]:
plt.plot( pca.components_[0,:])
plt.plot( vt[0,:])


In [ ]:
print(192*288)

In [ ]:
skoov=np.reshape( pca.components_ , (20,192,288) )

In [ ]:
plt.contour(skoov[14,:,:])